In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed

def scan_one_motif(mkey_pwm_tuple, seq):
    mkey, pwm = mkey_pwm_tuple
    scores = scan_motif_vector(seq, pwm)
    return mkey, scores

motifs = {k: pwm_to_logodds_Wx4(m) for k,m in read_meme(MEME).items()}
seq = "AAATGTGGTAACCCATGCGT"

results = {}
with ProcessPoolExecutor(max_workers=8) as exe:  # adjust workers
    futures = [exe.submit(scan_one_motif, (mkey, pwm), seq) for mkey, pwm in motifs.items()]
    for fut in as_completed(futures):
        mkey, scores = fut.result()
        results[mkey] = scores

print("Scanned", len(results), "motifs in parallel")

In [ ]:
from joblib import Parallel, delayed

def scan_one(mkey, pwm, seq):
    return mkey, scan_motif_vector(seq, pwm)

motifs = {k: pwm_to_logodds_Wx4(m) for k,m in read_meme(MEME).items()}
seq = "AAATGTGGTAACCCATGCGT"

results = Parallel(n_jobs=8)(
    delayed(scan_one)(mkey, pwm, seq) for mkey, pwm in motifs.items()
)

results = dict(results)  # convert to dictionary